<a href="https://colab.research.google.com/github/putrifpr/DA_Portfolio/blob/main/data_cleaning_ds_jobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [39]:
df = pd.read_csv('Uncleaned_DS_jobs.csv')
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [40]:
df.dtypes

,0
index,int64
Job Title,object
Salary Estimate,object
Job Description,object
Rating,float64
Company Name,object
Location,object
Headquarters,object
Size,object
Founded,int64


# **2. Data Cleaning**



*   Error Checking on Job Title


In [41]:
job_title_error = df['Job Title'].isnull().sum()
job_title_error

0



*   Checking on Salary Estimate



In [42]:
salary = df['Salary Estimate'].unique()
salary

array(['$137K-$171K (Glassdoor est.)', '$75K-$131K (Glassdoor est.)',
       '$79K-$131K (Glassdoor est.)', '$99K-$132K (Glassdoor est.)',
       '$90K-$109K (Glassdoor est.)', '$101K-$165K (Glassdoor est.)',
       '$56K-$97K (Glassdoor est.)', '$79K-$106K (Glassdoor est.)',
       '$71K-$123K (Glassdoor est.)', '$90K-$124K (Glassdoor est.)',
       '$91K-$150K (Glassdoor est.)', '$141K-$225K (Glassdoor est.)',
       '$145K-$225K(Employer est.)', '$79K-$147K (Glassdoor est.)',
       '$122K-$146K (Glassdoor est.)', '$112K-$116K (Glassdoor est.)',
       '$110K-$163K (Glassdoor est.)', '$124K-$198K (Glassdoor est.)',
       '$79K-$133K (Glassdoor est.)', '$69K-$116K (Glassdoor est.)',
       '$31K-$56K (Glassdoor est.)', '$95K-$119K (Glassdoor est.)',
       '$212K-$331K (Glassdoor est.)', '$66K-$112K (Glassdoor est.)',
       '$128K-$201K (Glassdoor est.)', '$138K-$158K (Glassdoor est.)',
       '$80K-$132K (Glassdoor est.)', '$87K-$141K (Glassdoor est.)',
       '$92K-$155K (Glassdo



*   Editing Salary Estimate Column (Remove anything beside the number)



In [43]:
nominal = r'\$(\d+)K-\$(\d+)K'

extracted = df['Salary Estimate'].str.extract(nominal)
print(extracted)

       0    1
0    137  171
1    137  171
2    137  171
3    137  171
4    137  171
..   ...  ...
667  105  167
668  105  167
669  105  167
670  105  167
671  105  167

[672 rows x 2 columns]


In [44]:
nominal = r'\$(\d+)K-\$(\d+)K'

df[['min_salary', 'max_salary']] = df['Salary Estimate'].str.extract(nominal).astype(int)

df['avg_salary'] = df.apply(lambda row: int((row['min_salary'] + row['max_salary']) / 2), axis=1)

df['Salary Estimate'] = df['min_salary'].astype(str) + '-' + df['max_salary'].astype(str)

df[['Salary Estimate', 'min_salary', 'max_salary', 'avg_salary']].head()

,Salary Estimate,min_salary,max_salary,avg_salary
0,137-171,137,171,154
1,137-171,137,171,154
2,137-171,137,171,154
3,137-171,137,171,154
4,137-171,137,171,154




*   Company Age



In [45]:
year_now = datetime.now().year
df['company_age'] = df['Founded'].map(lambda x: year_now - x if x > 0 else 0)

df['company_age'].head()

,company_age
0,31
1,56
2,43
3,24
4,26




*   Editing Revenue



In [46]:
# Fungsi cleaning Revenue
def revenue_clean(revenue):
  if pd.isna(revenue) or revenue == '-1': # jika data NaN atau -1
    return np.nan # maka isi NaN

# Menemukan angka dalam revenue
  numbers = re.findall(r'\d+', revenue)

# Memeriksa nilai revenue (million/billion)
  if 'million' in revenue.lower():
    bounds = [float(num)*1e6 for num in numbers] # untuk nilai million
  elif 'billion' in revenue.lower():
    bounds = [float(num)*1e9 for num in numbers] # untuk nilai billion
  else:
    return np.nan # jika tidak ada million/billion dalam revenue

# Menghitung rata-rata revenue
  if len(bounds) == 2:
    return np.mean(bounds) # rata-rata dua nilai di revenue
  elif len(bounds) == 1:
    return bounds[0] # jika hanya ada 1 nilai, tampilkan nilai revenue
  else:
    return np.nan

# Aplikasikan fungsi ke dalam kolom 'Revenue'
df['Average Revenue'] = df['Revenue'].apply(revenue_clean)
df['Average Revenue']

,Average Revenue
0,NaN
1,1.500000e+09
2,3.000000e+08
3,3.000000e+08
4,NaN
...,...
667,NaN
668,NaN
669,NaN
670,3.000000e+06




*   Job Description Qualify



In [47]:
df['Job Description'][0].split('\n')

['Description',
 '',
 'The Senior Data Scientist is responsible for defining, building, and improving statistical models to improve business processes and outcomes in one or more healthcare domains such as Clinical, Enrollment, Claims, and Finance. As part of the broader analytics team, Data Scientist will gather and analyze data to solve and address complex business problems and evaluate scenarios to make predictions on future outcomes and work with the business to communicate and support decision-making. This position requires strong analytical skills and experience in analytic methods including multivariate regressions, hierarchical linear models, regression trees, clustering methods and other complex statistical techniques.',
 '',
 'Duties & Responsibilities:',
 '',
 '• Develops advanced statistical models to predict, quantify or forecast various operational and performance metrics in multiple healthcare domains',
 '• Investigates, recommends, and initiates acquisition of new data 

In [48]:
df['python']   = df['Job Description'].map(lambda x: 1 if 'python'   in x.lower() else 0)
df['excel']    = df['Job Description'].map(lambda x: 1 if 'excel'    in x.lower() else 0)
df['hadoop']   = df['Job Description'].map(lambda x: 1 if 'hadoop'   in x.lower() else 0)
df['spark']    = df['Job Description'].map(lambda x: 1 if 'spark'    in x.lower() else 0)
df['aws']      = df['Job Description'].map(lambda x: 1 if 'aws'      in x.lower() else 0)
df['tableau']  = df['Job Description'].map(lambda x: 1 if 'tableau'  in x.lower() else 0)
df['big_data'] = df['Job Description'].map(lambda x: 1 if 'big data' in x.lower() else 0)
df['machine_learning']  = df['Job Description'].map(lambda x: 1 if 'machine learning'  in x.lower() else 0)
df['sql'] = df['Job Description'].map(lambda x: 1 if 'sql' in x.lower() else 0)
df['power_bi'] = df['Job Description'].map(lambda x: 1 if 'power bi' in x.lower() else 0)

In [71]:
df[['python','excel','hadoop','spark','aws','tableau','big_data','machine_learning','sql','power_bi']]

,python,excel,hadoop,spark,aws,tableau,big_data,machine_learning,sql,power_bi
0,0,0,0,0,1,0,0,1,0,0
1,0,0,1,0,0,0,1,1,1,0
2,1,1,0,0,1,0,0,1,0,0
3,1,1,0,0,1,0,0,1,1,0
4,1,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
667,1,1,1,0,0,1,1,1,1,1
668,0,0,0,0,0,0,0,0,0,0
669,1,1,1,0,0,1,0,1,1,1
670,1,0,1,1,0,0,1,1,1,0


In [77]:
# Mengganti \n\n dengan spasi
df['Job Description'] = df['Job Description'].str.replace('\n\n', ' ', regex=False)

# Mengganti \n dengan spasi (untuk sisa newline tunggal yang belum terganti)
df['Job Description'] = df['Job Description'].str.replace('\n', ' ', regex=False)

df['Job Description']

,Job Description
0,DescriptionThe Senior Data Scientist is respon...
1,"Secure our Nation, Ignite your FutureJoin the ..."
2,Overview Analysis Group is one of the largest ...
3,JOB DESCRIPTION:Do you have a passion for Data...
4,Data Scientist Affinity Solutions / Marketing ...
...,...
667,SummaryWe’re looking for a data scientist to d...
668,Job Description Become a thought leader within...
669,Join a thriving company that is changing the w...
670,100 Remote Opportunity As an AINLP Data Scient...


In [76]:
df['Job Description'][1]

"Secure our Nation, Ignite your FutureJoin the top Information Technology and Analytic professionals in the industry to make invaluable contributions to our national security on a daily basis. In this innovative, self-contained, Big Data environment, the ManTech team is responsible for everything from infrastructure, to application development, to data science, to advanced analytics and beyond. The team is diverse, the questions are thought-provoking, and the opportunities for growth and advancement are numerousThe successful candidate will possess a diverse range of data-focused skills and experience, both technical and analytical. They will have a strong desire and capability for problem solving, data analysis and troubleshooting, analytical thinking, and experimentation.Duties, Tasks & Responsibilities Working with large, complex, and disparate data sets Designing and implementing innovative ways to analyze and exploit the Sponsors data holdings Researching and reporting on a wide v



*   Company Rating



In [50]:
df['Rating'] = df['Rating'].map(lambda x: 0 if x<0 else x)

rating = df['Rating'].unique()
rating

array([3.1, 4.2, 3.8, 3.5, 2.9, 3.9, 4.4, 3.6, 4.5, 4.7, 3.7, 3.4, 4.1,
       3.2, 4.3, 2.8, 5. , 4.8, 3.3, 2.7, 2.2, 2.6, 4. , 2.5, 4.9, 2.4,
       0. , 2.3, 4.6, 3. , 2.1, 2. ])



*   Company Name


In [51]:
df['Company Name'] = df['Company Name'].map(lambda x: x.split('\n')[0])
df['Company Name']

,Company Name
0,Healthfirst
1,ManTech
2,Analysis Group
3,INFICON
4,Affinity Solutions
...,...
667,TRANZACT
668,JKGT
669,AccessHope
670,ChaTeck Incorporated




*   Competitor



In [52]:
sorted(df['Competitors'].unique())

['-1',
 'ACRT Services, Bartlett Tree Experts',
 'AT&T, Verizon',
 'Accenture, Deloitte, PwC',
 'Accenture, Northrop Grumman, Xerox',
 'Activision Blizzard, Electronic Arts',
 'Acxiom, Merkle, Epsilon (North Carolina)',
 'Adecco, Manpower',
 'Adecco, ManpowerGroup, Allegis Corporation',
 'Advisory Board, Booz Allen Hamilton, McKinsey & Company',
 'Amazon, Accenture, Microsoft',
 'Amazon, Apple',
 'AppDynamics, Datadog, Dynatrace',
 'Aquent, 24 Seven Talent',
 'Arbella Insurance, Safety Insurance',
 'Archibus, iOffice, Planon',
 'Battelle, General Atomics, SAIC',
 'Bechtel Jacobs, Black & Veatch, HNTB',
 'Booz Allen Hamilton, CACI International',
 'Booz Allen Hamilton, Deloitte, ERPi',
 'Booz Allen Hamilton, SAIC, LMI',
 'Bosch, Lear Corporation, Faurecia',
 'Braintree, Authorize.Net, PayPal',
 'Bromium, FireEye, Authentic8',
 'CDW, PCM, SHI International',
 'CGI (Nevada), Accenture, Deloitte',
 'CSC, ManTech, SAIC',
 'CUNA Mutual, SWBC, Overby-Seawell',
 'Cadence Design Systems, Synops

In [53]:
categorical_cols = ['Job Title', 'Salary Estimate', 'Job Description', 'Company Name', 'Location', 'Headquarters', 'Size', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors']

df[categorical_cols] = df[categorical_cols].replace('-1', 'na')
df[categorical_cols] = df[categorical_cols].replace('Unknown', 'na')



*   Location Editing



In [54]:
df['Location'].unique()

array(['New York, NY', 'Chantilly, VA', 'Boston, MA', 'Newton, MA',
       'Santa Barbara, CA', 'Cambridge, MA', 'Bedford, MA',
       'San Diego, CA', 'Chicago, IL', 'Herndon, VA', 'Saint Louis, MO',
       'Richland, WA', 'Northbrook, IL', 'Washington, DC', 'Remote',
       'Memphis, TN', 'Plano, TX', 'West Grove, PA', 'Phoenix, AZ',
       'Appleton, WI', 'Atlanta, GA', 'Orlando, FL', 'Lexington, MA',
       'McLean, VA', 'San Francisco, CA', 'Sheboygan, WI',
       'United States', 'Bothell, WA', 'Lincoln, NE', 'Overland Park, KS',
       'Santa Monica, CA', 'Portsmouth, NH', 'Ewing, NJ',
       'South San Francisco, CA', 'Palo Alto, CA', 'Bellevue, WA',
       'New Orleans, LA', 'Akron, OH', 'Fort Wayne, IN', 'Woburn, MA',
       'Carson, CA', 'Coral Gables, FL', 'Santa Clara, CA',
       'Brisbane, CA', 'Winter Park, FL', 'Redwood City, CA',
       'Peoria, IL', 'Ipswich, MA', 'Carmel, IN', 'Emeryville, CA',
       'Gaithersburg, MD', 'Longmont, CO', 'Austin, TX', 'Yakima, WA',
 

In [55]:
# Filter DataFrame untuk mencari baris yang tidak memiliki tanda koma di kolom 'Location'
df[~df['Location'].str.contains(',', na=False)]['Location'].unique()

array(['Remote', 'United States', 'Utah', 'New Jersey', 'Texas',
       'California'], dtype=object)

In [56]:
# Menyatakan Location dengan inisial
def assign_state(location):
    if location == "United States":
        return "Across US"
    elif location == "Utah":
        return "UT"
    elif location == "New Jersey":
        return "NJ"
    elif location == "Texas":
        return "TX"
    elif location == "Remote":
        return "Remote"
    elif location == "California":
        return "CA"
    else:
        # Mengambil State (inisial 2 huruf) di Location
        state = pd.Series(location).str.extract(',\s*(\w\w)$').iloc[0, 0]
        return state if pd.notna(state) else "Unknown"

# Membuat kolom baru 'job_state' dengan mengekstrak fungsi sebelumnya
df['job_state'] = df['Location'].apply(assign_state)

df[['Location', 'job_state']].head()

,Location,job_state
0,"New York, NY",NY
1,"Chantilly, VA",VA
2,"Boston, MA",MA
3,"Newton, MA",MA
4,"New York, NY",NY




*   Headquarters Editing



In [57]:
# Filter DataFrame untuk mencari baris yang tidak memiliki tanda koma di kolom 'Headquarters'
df[~df['Headquarters'].str.contains(',', na=False)]['Headquarters'].unique()

array(['na'], dtype=object)

In [58]:
# Membuat kolom baru 'job_state' dengan mengekstrak fungsi sebelumnya (assign_state)
df['hq_state'] = df['Headquarters'].apply(assign_state)

df[['Headquarters','hq_state']].head()

,Headquarters,hq_state
0,"New York, NY",NY
1,"Herndon, VA",VA
2,"Boston, MA",MA
3,"Bad Ragaz, Switzerland",Unknown
4,"New York, NY",NY


In [59]:
df['same_state'] = df.apply(lambda row: 1 if row['job_state'] == row['hq_state'] else 0, axis=1)

In [60]:
#Menampilkan State pada Job dan Headquarters
df[['job_state', 'hq_state', 'same_state']].head()

,job_state,hq_state,same_state
0,NY,NY,1
1,VA,VA,1
2,MA,MA,1
3,MA,Unknown,0
4,NY,NY,1




*   Job Type Require



In [61]:
# Membuat fungsi untuk menunjukkan tingkatan senior atau tidak pekerja yang dicari
def job_require(title) :
  if 'senior' in title.lower() or 'sr' in title.lower():
    return 1
  else:
    return 0

# Membuat kolom baru 'advance worker'
df['advance_worker'] = df['Job Title'].apply(job_require)
df[['Job Title','advance_worker']]

,Job Title,advance_worker
0,Sr Data Scientist,1
1,Data Scientist,0
2,Data Scientist,0
3,Data Scientist,0
4,Data Scientist,0
...,...,...
667,Data Scientist,0
668,Data Scientist,0
669,Data Scientist,0
670,Data Scientist,0


In [62]:
df

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,aws,tableau,big_data,machine_learning,sql,power_bi,job_state,hq_state,same_state,advance_worker
0,0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000 employees,1993,...,1,0,0,1,0,0,NY,NY,1,1
1,1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,...,0,0,1,1,1,0,VA,VA,1,0
2,2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,...,1,0,0,1,0,0,MA,MA,1,0
3,3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,...,1,0,0,1,1,0,MA,Unknown,0,0
4,4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,1998,...,0,0,0,1,1,0,NY,NY,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,667,Data Scientist,105-167,Summary\n\nWe’re looking for a data scientist ...,3.6,TRANZACT,"Fort Lee, NJ","Fort Lee, NJ",1001 to 5000 employees,1989,...,0,1,1,1,1,1,NJ,NJ,1,0
668,668,Data Scientist,105-167,Job Description\nBecome a thought leader withi...,0.0,JKGT,"San Francisco, CA",na,na,-1,...,0,0,0,0,0,0,CA,Unknown,0,0
669,669,Data Scientist,105-167,Join a thriving company that is changing the w...,0.0,AccessHope,"Irwindale, CA",na,na,-1,...,0,1,0,1,1,1,CA,Unknown,0,0
670,670,Data Scientist,105-167,100 Remote Opportunity As an AINLP Data Scient...,5.0,ChaTeck Incorporated,"San Francisco, CA","Santa Clara, CA",1 to 50 employees,-1,...,0,0,1,1,1,0,CA,CA,1,0


In [64]:
# Memindahkan posisi kolom

df.insert(26, 'machine_learning', df.pop('machine_learning'))
df.insert(27, 'sql', df.pop('sql'))
df.insert(28, 'power_bi', df.pop('power_bi'))

In [65]:
df.columns

Index(['index', 'Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'min_salary', 'max_salary', 'avg_salary', 'company_age',
       'Average Revenue', 'python', 'excel', 'hadoop', 'spark', 'aws',
       'tableau', 'machine_learning', 'sql', 'power_bi', 'big_data',
       'job_state', 'hq_state', 'same_state', 'advance_worker'],
      dtype='object')